In [2]:
from z3 import *
import numpy as np

#------------------------------------------------------------------------------
# Utils
#------------------------------------------------------------------------------

def maximum(x):
    m = x[0]
    for v in x[1:]:
        m = If(v > m, v, m)
    return m

#------------------------------------------------------------------------------
# Parameters
#------------------------------------------------------------------------------

def run_model_on_instance(file):
    with open(file) as f:
        m = int(next(f))
        n = int(next(f))
        l = [int(e) for e in next(f).split()]
        s = [int(e) for e in next(f).split()]
        D = np.genfromtxt(f, dtype=int).tolist()
    return m, n, l, s, D

#------------------------------------------------------------------------------
# Model
#------------------------------------------------------------------------------

def SMT(m, n, l, s, D):
    COURIERS = range(m)
    ITEMS = range(n)

    #------------------------------------------------------------------------------
    # Variables
    #------------------------------------------------------------------------------

    # A = Function('A', IntSort(), IntSort(), IntSort())
    # O = Function('O', IntSort(), IntSort(), IntSort())
    # dist = Function('dist', IntSort(), IntSort())

    A = [ [ Int("a_%s_%s" % (i+1, j+1)) for j in ITEMS ]
        for i in COURIERS ]
    
    O = [ [ Int("o_%s_%s" % (i+1, j+1)) for j in ITEMS ]
        for i in COURIERS ]

    dist = [ Int("dist_%s" % (i+1) )for i in COURIERS ]

    opt = Optimize()

    #------------------------------------------------------------------------------
    # Constraints
    #------------------------------------------------------------------------------

    # Total items size less than total couriers capacity
    opt.add(Sum([l[i] for i in COURIERS]) >= Sum([s[j] for j in ITEMS]))

    # Constraints to create A
    for i in COURIERS:
        for j in ITEMS:
            opt.add(A[i][j] >= 0)
    for i in COURIERS:
        assignment = [A[i][j] for j in ITEMS]
        opt.add(And(Sum(assignment) > 0, Sum(assignment) <= l[i]))
    for j in ITEMS:
        assignment = [A[i][j] for i in COURIERS]
        opt.add(And(Sum(assignment) == s[j], maximum(assignment) == s[j]))

    # Constraints to create O
    for i in COURIERS:
        for j in ITEMS:
            opt.add(If(A[i][j] == 0, O[i][j] == 0, O[i][j] > 0))
    for i in COURIERS:
        order_items = [If(O[i][j] > 0, O[i][j], 0) for j in ITEMS]
        non_zero_items = [If(order_items[j] != 0, order_items[j], -j) for j in ITEMS]
        count = Sum([If(non_zero_items[j] > 0, 1, 0) for j in ITEMS])
        opt.add(Distinct(non_zero_items))
        opt.add(And([And(0 <= order_items[j], order_items[j] <= count) for j in ITEMS]))
        opt.add(Sum([If(order_items[e] == 1, 1, 0) for e in ITEMS]) == 1)

    # Constraint to create dist
    for i in COURIERS:
        order_items = [O[i][j] for j in ITEMS]
        c = Sum([If(order_items[j] > 0, 1, 0) for j in ITEMS])
        dist_expr = Sum([
            Sum([
                If(And(order_items[j1] != 0, order_items[j2] - order_items[j1] == 1), D[j1][j2], 0)
                for j2 in ITEMS
            ])
            for j1 in ITEMS
        ])
        dist_expr += Sum([If(order_items[j0] == 1, D[n][j0], 0) for j0 in ITEMS])
        dist_expr += Sum([If(order_items[jn] == c, D[jn][n], 0) for jn in ITEMS])
        opt.add(dist[i] == dist_expr)

    #------------------------------------------------------------------------------
    # Objective
    #------------------------------------------------------------------------------

    # Declare objective variables
    obj = Int('obj')

    # Set objective function
    opt.add(obj == maximum([dist[i] for i in COURIERS]))

    #------------------------------------------------------------------------------
    # Search Strategy
    #------------------------------------------------------------------------------

    # Solve the SMT problem
    opt.minimize(obj)

    print(f"{m}\n{l}\n{n}\n{s}\n\n\n")

    if opt.check() == sat:
        model = opt.model()
        result_A = [ [ model.evaluate(A[i][j]) for j in ITEMS ] 
            for i in COURIERS ]
        result_O = [ [ model.evaluate(O[i][j]) for j in ITEMS ] 
            for i in COURIERS ]
        result_dist = [ model.evaluate(dist[i]) for i in COURIERS ]
        result_objective = model.evaluate(obj)
        for i in COURIERS:
            print_matrix(result_A[i])
        print()
        for i in COURIERS:
            print_matrix(result_O[i])
        print()
        print_matrix(result_dist)
        print(f"\nobjective: {result_objective}")
    else:
        print ("failed to solve")

SMT(*run_model_on_instance("./instances/inst03.dat"))

3
[15, 10, 7]
7
[3, 2, 6, 8, 5, 4, 4]



[0, 2, 0, 8, 5, 0, 0]
[0, 0, 6, 0, 0, 4, 0]
[3, 0, 0, 0, 0, 0, 4]

[0, 3, 0, 2, 1, 0, 0]
[0, 0, 1, 0, 0, 2, 0]
[2, 0, 0, 0, 0, 0, 1]

[12, 10, 12]

objective: 12
